In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("haha").getOrCreate()

In [ ]:
boom = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "kafka.nadzieja.test:9092")
    .option("subscribe", "dictionary")
    .load()
)

In [ ]:
boom1 = boom.selectExpr("from_json(cast(value as string), 'nama STRING, alasan STRING') as message")
boom2 = boom1.select("message.nama")

In [ ]:
boomWriter = (
    boom2.filter("nama is not null")
    .selectExpr("cast(null as string) as key", "to_json(struct(nama)) as value")
    .writeStream.format("kafka")
    .option("topic", "people")
    .option("kafka.bootstrap.servers", "kafka.nadzieja.test:9092")
)

boomWriter.option("checkpointLocation", "checkpoint").start()